# The goal is to create one master Dataframe/CSV with each of these metrics for BiggerScore V1
- 5-Year Household Formation
- 5-Year Population Growth
- 5-Year Job Growth
- 5-Year Wage Growth
- Unemployment Rate
- SFH Units Permitted
- MF Units Permitted
- Median Price
- Median Rent
- Rent-Price Ratio
- Average Insurance
- Median Property Taxes
- Vacancy Rate
- 1-Year Appreciation
- 1-Year Rent Growth
- Population Size Category
- Landlord Friendliness

In [638]:
# Import modules
import pandas as pd
import numpy as np
from numpy import log as ln
from numpy import log10
from numpy_financial import pmt
import os
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd
from functools import reduce

# Set up Pandas defaults
pd.options.display.float_format = '{:.4f}'.format
pd.set_option("display.max_columns", None)

# Define helper function to create directory
def create_folder(the_path):
    "Create directory if nonexistent."
    if not os.path.isdir(the_path):
        os.mkdir(the_path)

create_folder('outputs')

census_end_year = int(input("What year was the most recent ACS 5-Year census data pulled from? "))
acs_1_year = int(input("What year was the most recent ACS 1-Year census data pulled from? "))

In [639]:
def clean_BLS_msa_names(dataframe):
    """
    This functions standardizes the MSA names
    between the BLS dataset and the Zillow
    datasets by only taking the first city
    within a BLS-MSA with a "-" hyphenate, 
    and the first state with a "-" hyphenate.
    
    For example: 'Houston-The Woodlands-Sugar Land, TX'
    will be turned into just 'Houston, TX' and
    'Cincinnati, OH-KY-IN' will be turned into
    just 'Cincinnati, OH'.

    An exception will be made for 'Wildwood-The Villages, FL', 
    which will be reduced to just 'The Villages, FL'.
    """
    
    df = dataframe.copy()
    
    # Get the state column
    df['state'] = df['msa_name'].str.split(',').str[1].str.strip().str.strip("*")
    df['state'] = df['state'].str.split('-').str[0].str.strip()
    
    # Get the first city name
    df['city'] = df['msa_name'].str.split(',').str[0].str.strip()
    df['city'] = df['city'].str.split('-').str[0].str.strip()
    df['city'] = df['city'].str.split('/').str[0].str.strip()

    # Remove 'NECTA' from city name
    df['state'] = df['state'].str.replace(" NECTA", "", regex=False)
    
    # Get msa name
    df['msa_name'] = df['city'] + ", " + df['state']

    ### Create exceptions
    df.loc[df['msa_name']=='Wildwood, FL', 'msa_name'] = 'The Villages, FL'
    
    return df

### Get 5-year Household formation for all MSAs

In [640]:
households = pd.read_csv(
    "../downloaded_datasets/cleaned_census_api_files/standardized/standardized_msa/households_msa_from_county_sum.csv", 
    dtype={'msa_code':str})

households['5-Year Household Growth'] = (households[f'{census_end_year}'] - households[f'{census_end_year - 5}']) / households[f'{census_end_year - 5}']
households['1-Year Household Growth'] = (households[f'{census_end_year}'] - households[f'{census_end_year - 1}']) / households[f'{census_end_year - 1}']


# Only keep certian columns
households = households[['msa_name','msa_code','5-Year Household Growth','1-Year Household Growth']]

# EDA
households.sort_values('5-Year Household Growth', ascending=False)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth
58,"Austin-Round Rock-San Marcos, TX",12420,0.2500,0.0501
589,"Nantucket, MA",34880,0.2388,0.1558
903,"Wildwood-The Villages, FL",45540,0.2281,0.0466
732,"St. George, UT",41100,0.2246,0.0452
32,"Andrews, TX",11380,0.2198,0.0543
...,...,...,...,...
294,"Forrest City, AR",22620,-0.1312,-0.0054
103,"Borger, TX",14420,-0.1395,-0.0247
545,"Middlesborough, KY",33180,-0.1453,-0.0028
506,"Magnolia, AR",31620,-0.1618,0.0075


### Get 5-Year Population Growth for MSAs

In [641]:
population = pd.read_csv(
    "../downloaded_datasets/cleaned_census_api_files/standardized/standardized_msa/population_msa_from_county_sum.csv", 
    dtype={'msa_code':str})
# population = population[~population['msa_code'].isin(problem_msa_codes)] # Remove problem MSAs
population['5-Year Population Growth'] = (population[f'{census_end_year}'] - population[f'{census_end_year - 5}']) / population[f'{census_end_year - 5}']
population['1-Year Population Growth'] = (population[f'{census_end_year}'] - population[f'{census_end_year - 1}']) / population[f'{census_end_year - 1}']


# Only keep certian columns
population = population[['msa_name','msa_code',f'{census_end_year}',
                         '5-Year Population Growth','1-Year Population Growth']]

# Rename
population.rename(columns={f'{census_end_year}':'Population'}, inplace=True)

# EDA
population.sort_values('5-Year Population Growth', ascending=False).head(20)

,msa_name,msa_code,Population,5-Year Population Growth,1-Year Population Growth
701,"Rexburg, ID",39940,66006.0000,0.2909,0.0258
589,"Nantucket, MA",34880,14065.0000,0.2889,0.0196
412,"Jefferson, GA",27600,77033.0000,0.2064,0.0433
146,"Cedar City, UT",16260,58068.0000,0.1972,0.0399
641,"Pahrump, NV",37220,51698.0000,0.1941,0.0320
872,"Vineyard Haven, MA",47240,20543.0000,0.1892,0.0131
107,"Bozeman, MT",14580,119685.0000,0.1881,0.0254
905,"Williston, ND",48780,39076.0000,0.1871,0.0160
732,"St. George, UT",41100,183297.0000,0.1782,0.0383
336,"Greeley, CO",24540,331466.0000,0.1601,0.0280


### Get Vacancy Rate

In [642]:
vacancy = pd.read_csv(
    "../downloaded_datasets/cleaned_census_api_files/standardized/standardized_msa/vacancy_rate_msa_from_county_sum.csv", 
    dtype={'msa_code':str})
vacancy.rename(columns={f'{census_end_year}':'Vacancy_Rate'}, inplace=True)
vacancy = vacancy[['msa_name', 'msa_code', 'Vacancy_Rate']]
vacancy

,msa_name,msa_code,Vacancy_Rate
0,"Aberdeen, SD",10100,0.1166
1,"Aberdeen, WA",10140,0.1840
2,"Abilene, TX",10180,0.1270
3,"Ada, OK",10220,0.1505
4,"Adrian, MI",10300,0.1148
...,...,...,...
923,"Youngstown-Warren, OH",49660,0.0990
924,"Yuba City, CA",49700,0.0557
925,"Yuma, AZ",49740,0.1953
926,"Zanesville, OH",49780,0.0978


### Read in Price and Rent from Census ACS 1-Year

In [643]:
### Read in Price
census_price = pd.read_csv(
    "../downloaded_datasets/cleaned_census_api_files/msa_data/median_price_msa_acs1.csv",
    dtype={'msa_code':str})

census_price['5-Year Price Growth'] = (census_price[f'{acs_1_year}'] - census_price[f'{acs_1_year - 5}']) / census_price[f'{acs_1_year - 5}']

# Only keep certian columns
census_price = census_price[['msa_code',f'{acs_1_year}','5-Year Price Growth']]

# Rename
census_price.rename(columns={f'{acs_1_year}':'ACS_1_Year_Median_Price'}, inplace=True)

# EDA
census_price.sort_values('5-Year Price Growth', ascending=False).head(20)

,msa_code,ACS_1_Year_Median_Price,5-Year Price Growth
156,25980,235100.0000,1.0972
278,38540,341800.0000,1.0615
157,26140,278800.0000,1.0262
75,17660,556500.0000,1.0127
328,42700,208900.0000,0.9745
117,22220,342100.0000,0.9706
163,26820,386100.0000,0.8797
370,47460,429700.0000,0.8530
195,29460,296300.0000,0.8369
188,28940,320700.0000,0.8253


In [644]:
### Read in Rent
census_rent = pd.read_csv(
    "../downloaded_datasets/cleaned_census_api_files/msa_data/median_rent_msa_acs1.csv",
    dtype={'msa_code':str})

census_rent['5-Year Rent Growth'] = (census_rent[f'{acs_1_year}'] - census_rent[f'{acs_1_year - 5}']) / census_rent[f'{acs_1_year - 5}']

# Only keep certian columns
census_rent = census_rent[['msa_code',f'{acs_1_year}','5-Year Rent Growth']]

# Rename
census_rent.rename(columns={f'{acs_1_year}':'ACS_1_Year_Median_Rent'}, inplace=True)

# EDA
census_rent.sort_values('5-Year Rent Growth', ascending=False).head(20)

,msa_code,ACS_1_Year_Median_Rent,5-Year Rent Growth
305,41100,1529.0000,0.7257
275,38240,1203.0000,0.6896
274,38060,1605.0000,0.6806
249,34940,1867.0000,0.6420
43,14260,1379.0000,0.6320
47,14580,1664.0000,0.6266
211,30860,1054.0000,0.6067
268,37340,1450.0000,0.6022
349,45300,1558.0000,0.5979
85,18880,1458.0000,0.5952


In [645]:
### Merge the two and create a rent-price ratio
census_price_rent = census_price.merge(census_rent, how='inner', on='msa_code')

# Create rent-rpice ratio
census_price_rent['ACS_1_Year_Rent-Price_Ratio'] = 100 * census_price_rent['ACS_1_Year_Median_Rent'] / census_price_rent['ACS_1_Year_Median_Price']

census_price_rent


,msa_code,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio
0,10180,180400.0000,0.5096,985.0000,0.3643,0.5460
1,10380,136500.0000,0.4368,410.0000,0.2202,0.3004
2,10420,216000.0000,0.3891,819.0000,0.2133,0.3792
3,10500,164100.0000,0.4665,617.0000,0.1797,0.3760
4,10540,398300.0000,0.5663,1181.0000,0.3927,0.2965
...,...,...,...,...,...,...
388,49420,318900.0000,0.7099,933.0000,0.4136,0.2926
389,49620,257600.0000,0.4077,949.0000,0.2842,0.3684
390,49660,149700.0000,0.3051,626.0000,0.1636,0.4182
391,49700,398200.0000,0.3498,1114.0000,0.3619,0.2798


### Read in total units

In [646]:
# Read in Total Units
total_units = pd.read_csv("../downloaded_datasets/cleaned_census_api_files/standardized/standardized_msa/total_units_msa_from_county_sum.csv",
                          dtype={'msa_code':str})
total_units = total_units[['msa_code',f'{census_end_year}']]
total_units.rename(columns={f'{census_end_year}':'Total_Units'}, inplace=True)
total_units

,msa_code,Total_Units
0,10100,19896.0000
1,10140,36204.0000
2,10180,74540.0000
3,10220,17424.0000
4,10300,43634.0000
...,...,...
923,49660,202778.0000
924,49700,64186.0000
925,49740,92808.0000
926,49780,38348.0000


### Merge all census datasets

In [647]:
census = households.merge(population, how='left', on=['msa_name','msa_code'])
census = census.merge(vacancy, how='left', on=['msa_name', 'msa_code'])
census = census.merge(census_price_rent, how='left', on=['msa_code'])
census = census.merge(total_units, how='left', on=['msa_code'])

# Prep for BLS merge
census['msa_name_original'] = census['msa_name']
census = clean_BLS_msa_names(census)

census

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,state,city
0,"Aberdeen, SD",10100,-0.0184,-0.0063,42292.0000,-0.0074,-0.0043,0.1166,NaN,NaN,NaN,NaN,NaN,19896.0000,"Aberdeen, SD",SD,Aberdeen
1,"Aberdeen, WA",10140,0.0524,0.0076,75672.0000,0.0590,0.0125,0.1840,NaN,NaN,NaN,NaN,NaN,36204.0000,"Aberdeen, WA",WA,Aberdeen
2,"Abilene, TX",10180,0.0780,0.0116,176656.0000,0.0453,0.0081,0.1270,180400.0000,0.5096,985.0000,0.3643,0.5460,74540.0000,"Abilene, TX",TX,Abilene
3,"Ada, OK",10220,0.0199,0.0135,38116.0000,-0.0045,0.0004,0.1505,NaN,NaN,NaN,NaN,NaN,17424.0000,"Ada, OK",OK,Ada
4,"Adrian, MI",10300,0.0134,-0.0011,99263.0000,0.0069,-0.0008,0.1148,NaN,NaN,NaN,NaN,NaN,43634.0000,"Adrian, MI",MI,Adrian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,"Youngstown, OH",49660,-0.0111,0.0023,429728.0000,-0.0126,-0.0035,0.0990,149700.0000,0.3051,626.0000,0.1636,0.4182,202778.0000,"Youngstown-Warren, OH",OH,Youngstown
924,"Yuba City, CA",49700,0.0437,0.0171,180806.0000,0.0621,0.0074,0.0557,398200.0000,0.3498,1114.0000,0.3619,0.2798,64186.0000,"Yuba City, CA",CA,Yuba City
925,"Yuma, AZ",49740,0.0420,0.0270,204374.0000,0.0005,0.0070,0.1953,246900.0000,0.7560,878.0000,0.2688,0.3556,92808.0000,"Yuma, AZ",AZ,Yuma
926,"Zanesville, OH",49780,0.0073,0.0185,86393.0000,0.0054,0.0005,0.0978,NaN,NaN,NaN,NaN,NaN,38348.0000,"Zanesville, OH",OH,Zanesville


### Create a cateogry for population size

In [648]:
census['Population_Size_Category'] = np.where(
    census['Population']<100_000, 1,
    np.where(
        census['Population']<250_000, 2,
        np.where(
            census['Population']<500_000, 3,
            np.where(census['Population']<1_000_000, 4, 5)
        )
    )
)

census

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,state,city,Population_Size_Category
0,"Aberdeen, SD",10100,-0.0184,-0.0063,42292.0000,-0.0074,-0.0043,0.1166,NaN,NaN,NaN,NaN,NaN,19896.0000,"Aberdeen, SD",SD,Aberdeen,1
1,"Aberdeen, WA",10140,0.0524,0.0076,75672.0000,0.0590,0.0125,0.1840,NaN,NaN,NaN,NaN,NaN,36204.0000,"Aberdeen, WA",WA,Aberdeen,1
2,"Abilene, TX",10180,0.0780,0.0116,176656.0000,0.0453,0.0081,0.1270,180400.0000,0.5096,985.0000,0.3643,0.5460,74540.0000,"Abilene, TX",TX,Abilene,2
3,"Ada, OK",10220,0.0199,0.0135,38116.0000,-0.0045,0.0004,0.1505,NaN,NaN,NaN,NaN,NaN,17424.0000,"Ada, OK",OK,Ada,1
4,"Adrian, MI",10300,0.0134,-0.0011,99263.0000,0.0069,-0.0008,0.1148,NaN,NaN,NaN,NaN,NaN,43634.0000,"Adrian, MI",MI,Adrian,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,"Youngstown, OH",49660,-0.0111,0.0023,429728.0000,-0.0126,-0.0035,0.0990,149700.0000,0.3051,626.0000,0.1636,0.4182,202778.0000,"Youngstown-Warren, OH",OH,Youngstown,3
924,"Yuba City, CA",49700,0.0437,0.0171,180806.0000,0.0621,0.0074,0.0557,398200.0000,0.3498,1114.0000,0.3619,0.2798,64186.0000,"Yuba City, CA",CA,Yuba City,2
925,"Yuma, AZ",49740,0.0420,0.0270,204374.0000,0.0005,0.0070,0.1953,246900.0000,0.7560,878.0000,0.2688,0.3556,92808.0000,"Yuma, AZ",AZ,Yuma,2
926,"Zanesville, OH",49780,0.0073,0.0185,86393.0000,0.0054,0.0005,0.0978,NaN,NaN,NaN,NaN,NaN,38348.0000,"Zanesville, OH",OH,Zanesville,1


### Now read in the BLS metrics

In [649]:
job_growth = pd.read_csv("../downloaded_datasets/bls_msa_job_growth.csv", dtype={'msa_code':str})

# Get most recent date
job_growth['date'] = pd.to_datetime(job_growth['date'])
most_recent_date = job_growth['date'].max()

# Get date 5 years ago
five_yrs_ago = most_recent_date - pd.DateOffset(years=5)

# Only keep relevant dates
job_growth = job_growth[(job_growth['date']==most_recent_date) |
                        (job_growth['date']==five_yrs_ago) ]

# Sort again
job_growth = job_growth.sort_values(['msa_code','date'], ascending=[True, True])

# Calculate percent change and only keep the most recent date
job_growth['5-Year Job Growth'] = job_growth['value'].pct_change()
job_growth = job_growth[job_growth['date']==most_recent_date]

job_growth = job_growth[['msa_code','msa_name','value','5-Year Job Growth']]

# Rename
job_growth = job_growth.rename(columns={'value':'Jobs'})
job_growth = clean_BLS_msa_names(job_growth)

job_growth.head(10)


,msa_code,msa_name,Jobs,5-Year Job Growth,state,city
0,10180,"Abilene, TX",78200.0000,0.0831,TX,Abilene
128,10380,"Aguadilla, PR",59000.0000,0.1546,PR,Aguadilla
256,10420,"Akron, OH",336900.0000,-0.0068,OH,Akron
384,10500,"Albany, GA",63100.0000,0.0064,GA,Albany
512,10540,"Albany, OR",48200.0000,0.0478,OR,Albany
640,10580,"Albany, NY",472100.0000,0.0013,NY,Albany
768,10740,"Albuquerque, NM",423900.0000,0.0627,NM,Albuquerque
896,10780,"Alexandria, LA",61800.0000,-0.0032,LA,Alexandria
1024,10900,"Allentown, PA",399800.0000,0.0577,PA,Allentown
1152,11020,"Altoona, PA",62100.0000,0.0065,PA,Altoona


### Read in Wage Growth

In [650]:
wage_growth = pd.read_csv("../downloaded_datasets/bls_msa_average_wage.csv", dtype={'msa_code':str})

# Get most recent date
wage_growth['date'] = pd.to_datetime(wage_growth['date'])
most_recent_date = wage_growth['date'].max()

# Get date 5 years ago
five_yrs_ago = most_recent_date - pd.DateOffset(years=5)

# Only keep relevant dates
wage_growth = wage_growth[(wage_growth['date']==most_recent_date) |
                        (wage_growth['date']==five_yrs_ago) ]

# Sort again
wage_growth = wage_growth.sort_values(['msa_code','date'], ascending=[True, True])

# Calculate percent change and only keep the most recent date
wage_growth['5-Year Income Growth'] = wage_growth['value'].pct_change()
wage_growth = wage_growth[wage_growth['date']==most_recent_date]

wage_growth = wage_growth[['msa_code','value','5-Year Income Growth']]

# Rename
wage_growth = wage_growth.rename(columns={'value':'Income'})

wage_growth.sort_values("5-Year Income Growth", ascending=False).head(10)


,msa_code,Income,5-Year Income Growth
13666,20940,50214.3200,0.5756
42308,43420,55651.4400,0.5678
23010,27980,65271.4400,0.5603
16738,23540,58091.8000,0.5427
22498,27740,53904.2400,0.5226
4834,13900,72016.8800,0.5194
48580,48900,58167.2000,0.4998
11490,19180,59978.8800,0.4853
18018,24500,58626.8800,0.4815
3682,12980,57369.0000,0.4788


### Read in Unemployment

In [651]:
unemployment = pd.read_csv("../downloaded_datasets/bls_msa_unemployment.csv", dtype={'msa_code':str})

unemployment['date'] = pd.to_datetime(unemployment['date'])
unemployment = unemployment[unemployment['date']==unemployment['date'].max()]

unemployment = unemployment[['msa_code','value']]
unemployment.rename(columns={'value':'Unemployment_Rate'}, inplace=True)
unemployment

,msa_code,Unemployment_Rate
0,10180,3.8000
128,10380,7.5000
256,10420,4.3000
384,10500,4.9000
512,10540,4.5000
...,...,...
49152,49420,6.2000
49280,49620,4.0000
49408,49660,5.0000
49536,49700,7.3000


### Merge all BLS data together

In [652]:
bls_data = job_growth.merge(wage_growth, how='inner', on='msa_code')
bls_data = bls_data.merge(unemployment, how='inner', on='msa_code')
bls_data.drop(columns=['state','city'], inplace=True)
bls_data

,msa_code,msa_name,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate
0,10180,"Abilene, TX",78200.0000,0.0831,53747.7200,0.3829,3.8000
1,10380,"Aguadilla, PR",59000.0000,0.1546,27089.4000,0.1748,7.5000
2,10420,"Akron, OH",336900.0000,-0.0068,58651.8400,0.3022,4.3000
3,10500,"Albany, GA",63100.0000,0.0064,40566.2400,0.0164,4.9000
4,10540,"Albany, OR",48200.0000,0.0478,58515.6000,0.3933,4.5000
...,...,...,...,...,...,...,...
384,76600,"Pittsfield, MA",41000.0000,-0.0398,49026.6400,0.0749,4.2000
385,76750,"Portland, ME",224700.0000,0.0298,57711.6800,0.2344,2.2000
386,77200,"Providence, RI",605700.0000,0.0103,58959.1600,0.2799,5.3000
387,78100,"Springfield, MA",332100.0000,-0.0104,55604.6400,0.1018,5.0000


### Merge Census data with BLS data

In [653]:
census_bls = census.merge(bls_data, how='left', on=['msa_name'])
census_bls.rename(columns={
    'msa_code_x':'msa_code_census',
    'msa_code_y':'msa_code_bls'
}, inplace=True)
census_bls

,msa_name,msa_code_census,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,state,city,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate
0,"Aberdeen, SD",10100,-0.0184,-0.0063,42292.0000,-0.0074,-0.0043,0.1166,NaN,NaN,NaN,NaN,NaN,19896.0000,"Aberdeen, SD",SD,Aberdeen,1,NaN,NaN,NaN,NaN,NaN,NaN
1,"Aberdeen, WA",10140,0.0524,0.0076,75672.0000,0.0590,0.0125,0.1840,NaN,NaN,NaN,NaN,NaN,36204.0000,"Aberdeen, WA",WA,Aberdeen,1,NaN,NaN,NaN,NaN,NaN,NaN
2,"Abilene, TX",10180,0.0780,0.0116,176656.0000,0.0453,0.0081,0.1270,180400.0000,0.5096,985.0000,0.3643,0.5460,74540.0000,"Abilene, TX",TX,Abilene,2,10180,78200.0000,0.0831,53747.7200,0.3829,3.8000
3,"Ada, OK",10220,0.0199,0.0135,38116.0000,-0.0045,0.0004,0.1505,NaN,NaN,NaN,NaN,NaN,17424.0000,"Ada, OK",OK,Ada,1,NaN,NaN,NaN,NaN,NaN,NaN
4,"Adrian, MI",10300,0.0134,-0.0011,99263.0000,0.0069,-0.0008,0.1148,NaN,NaN,NaN,NaN,NaN,43634.0000,"Adrian, MI",MI,Adrian,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,"Youngstown, OH",49660,-0.0111,0.0023,429728.0000,-0.0126,-0.0035,0.0990,149700.0000,0.3051,626.0000,0.1636,0.4182,202778.0000,"Youngstown-Warren, OH",OH,Youngstown,3,49660,211300.0000,-0.0227,44661.2400,0.0732,5.0000
924,"Yuba City, CA",49700,0.0437,0.0171,180806.0000,0.0621,0.0074,0.0557,398200.0000,0.3498,1114.0000,0.3619,0.2798,64186.0000,"Yuba City, CA",CA,Yuba City,2,49700,52900.0000,0.1328,55434.0800,0.2713,7.3000
925,"Yuma, AZ",49740,0.0420,0.0270,204374.0000,0.0005,0.0070,0.1953,246900.0000,0.7560,878.0000,0.2688,0.3556,92808.0000,"Yuma, AZ",AZ,Yuma,2,49740,61600.0000,0.1039,52416.5200,0.4144,16.3000
926,"Zanesville, OH",49780,0.0073,0.0185,86393.0000,0.0054,0.0005,0.0978,NaN,NaN,NaN,NaN,NaN,38348.0000,"Zanesville, OH",OH,Zanesville,1,NaN,NaN,NaN,NaN,NaN,NaN


### Now merge Building Permit Survey (Supply) data

In [654]:
### Read in permitted units tidy for all housing types 
### (if you don't see it in your repo, you need to run 
### "helper_functions/permit_cleaning.py" to clean and create the file)

# We will use research conducted by the Federal Reserve Bank of Atlanta, and use
# an approximate 24 months from permit to completion as a lower bound for multifamly
# completions. Single-Family likely takes less time, but that depends on location. 
# In addition, 50-100 units takes approximately 30 months for completion on average.
# Due to the large variance, I will be using an arbitrary 2-year period from
# permit to completion. Every market will have their own average and will be different
# from project-to-project, so the 2-year time-to-completion will be a rule-of-thumb for simplicity.

permits = pd.read_csv("../datasets_manual_download/building_permits/cleaned/permitted_all_housing_types_tidy.csv",
                      dtype={'msa_code':str})

# Permit end year will be more recent than the census end year
previous_year = census_end_year + 1

# Only keep the most recent year
permits = permits[permits['year']==previous_year]
permits.rename(columns={'year':'permit_data_for_year'}, inplace=True)
permits

,msa_name,msa_code,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi
9,"Abilene, TX",10180,2023,342.0000,56.0000,56.0000,64.0000,462.0000,74.0260,12.1212,12.1212,13.8528
19,"Akron, OH",10420,2023,709.0000,4.0000,107.0000,124.0000,940.0000,75.4255,0.4255,11.3830,13.1915
29,"Albany, GA",10500,2023,162.0000,2.0000,6.0000,15.0000,183.0000,88.5246,1.0929,3.2787,8.1967
39,"Albany, OR",10540,2023,239.0000,52.0000,59.0000,79.0000,377.0000,63.3952,13.7931,15.6499,20.9549
49,"Albany, NY",10580,2023,976.0000,40.0000,95.0000,836.0000,1907.0000,51.1799,2.0975,4.9816,43.8385
...,...,...,...,...,...,...,...,...,...,...,...,...
3729,"Yakima, WA",49420,2023,453.0000,58.0000,139.0000,58.0000,650.0000,69.6923,8.9231,21.3846,8.9231
3739,"York, PA",49620,2023,983.0000,14.0000,38.0000,146.0000,1167.0000,84.2331,1.1997,3.2562,12.5107
3749,"Youngstown, OH",49660,2023,282.0000,0.0000,8.0000,0.0000,290.0000,97.2414,0.0000,2.7586,0.0000
3759,"Yuba City, CA",49700,2023,589.0000,20.0000,20.0000,0.0000,609.0000,96.7159,3.2841,3.2841,0.0000


In [655]:
permits[permits['msa_name'].str.contains("Villages")]

,msa_name,msa_code,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi
3399,"The Villages, FL",45540,2023,3339.0000,0.0000,0.0000,0.0000,3339.0000,100.0000,0.0000,0.0000,0.0000


In [656]:
census_bls[census_bls['msa_name_original'].str.contains("Villages")]

,msa_name,msa_code_census,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,state,city,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate
903,"The Villages, FL",45540,0.2281,0.0466,131832.0000,0.1291,0.0353,0.1640,NaN,NaN,NaN,NaN,NaN,76923.0000,"Wildwood-The Villages, FL",FL,Wildwood,2,45540,41200.0000,0.2997,41499.1200,0.0161,5.2000


### Now merge it with the main dataset

In [657]:
# census_bls_permits = census_bls.merge(permits, how='left', left_on='msa_name', right_on='msa_name')
census_bls_permits = census_bls.merge(permits, how='left', on='msa_name')

# Create percent of total units column
census_bls_permits['Permits_as_Percent_of_Total_Units'] = census_bls_permits['total_unit_permits'] / census_bls_permits['Total_Units']
census_bls_permits = census_bls_permits.sort_values('Permits_as_Percent_of_Total_Units', ascending=False).reset_index(drop=True)

# Another cleaning
census_bls_permits.drop(columns=['state','city','msa_code'], inplace=True)
census_bls_permits.rename(columns={'msa_code_census':'msa_code'}, inplace=True)

census_bls_permits.head(3)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units
0,"Salisbury, MD",41540,0.0550,0.0153,128487.0000,0.0053,0.0034,0.1162,248500.0000,0.0720,1021.0000,0.2391,0.4109,54646.0000,"Salisbury, MD",2,41540,180800.0000,0.0290,47292.9600,0.2144,3.8000,2023.0000,4177.0000,90.0000,110.0000,607.0000,4894.0000,85.3494,1.8390,2.2477,12.4029,0.0896
1,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,202800.0000,0.1118,44395.5200,0.3483,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637
2,"The Villages, FL",45540,0.2281,0.0466,131832.0000,0.1291,0.0353,0.1640,NaN,NaN,NaN,NaN,NaN,76923.0000,"Wildwood-The Villages, FL",2,45540,41200.0000,0.2997,41499.1200,0.0161,5.2000,2023.0000,3339.0000,0.0000,0.0000,0.0000,3339.0000,100.0000,0.0000,0.0000,0.0000,0.0434


### Now merge with market pulse data (price and rent), HPI, and RPI data from HouseCanary
(In Snowflake, I simply queried the `MARKETDATA_DB.RAW.HOUSECANARY_MSA_HISTORICAL_MARKET_PULSE` and `MARKETDATA_DB.RAW.HOUSECANARY_MSA_HISTORICAL_RENTAL_MARKET_PULSE` tables where the DATE_ACCESSED was from the most recent API pull, then I simply downloaded those queries to csv's. You'll see their name and folder location in the code below.)

I also queried `MARKETDATA_DB.RAW.HOUSECANARY_MSA_HPI_FORECAST` and `MARKETDATA_DB.RAW.HOUSECANARY_MSA_RPI_FORECAST` to get the HPI and RPI data as well.

In [658]:
# Read in median price
hc_market = pd.read_csv(
    "../downloaded_datasets/transferred_from_house_canary_repo/msa_outputs/msa_historical_market_pulse.csv",
    dtype={'MSA_CODE':str})
hc_market = hc_market[['MSA_CODE','PRICECLOSEDMEDIAN']]
hc_market.rename(
    columns={'MSA_CODE':'msa_code', 'PRICECLOSEDMEDIAN':'housecanary_median_price'}, inplace=True)
hc_market = hc_market.dropna()

# Read in median rent
hc_rent_market = pd.read_csv(
    "../downloaded_datasets/transferred_from_house_canary_repo/msa_outputs/msa_historical_rental_market_pulse.csv",
    dtype={'MSA_CODE':str})
for col in hc_rent_market.columns:
    hc_rent_market.rename(columns={col:col.lower()}, inplace=True)

# Only keep markets with enough listing data
hc_rent_market = hc_rent_market[
    hc_rent_market['listingsonmarketcount']>=hc_rent_market['listingsonmarketcount'].quantile(0.25)]

# Only keep specific columns
hc_rent_market = hc_rent_market[['msa_code',
                                #  'msa_name',
                                 'priceonmarketmedian']]
hc_rent_market.rename(columns={'priceonmarketmedian':'housecanary_median_rent'}, inplace=True)

hc_rent_market

# Now merge the two
hc_price_and_rent = hc_market.merge(hc_rent_market, how='inner', on='msa_code')
hc_price_and_rent['housecanary_rentpriceratio'] = 100 * hc_price_and_rent['housecanary_median_rent'] / hc_price_and_rent['housecanary_median_price']

hc_price_and_rent.sort_values('housecanary_rentpriceratio', ascending=False)


,msa_code,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio
77,46540,223650.0000,2300.0000,1.0284
102,15380,250000.0000,2525.0000,1.0100
63,44100,205000.0000,1998.0000,0.9746
53,13780,206250.0000,2000.0000,0.9697
150,22420,213450.0000,2050.0000,0.9604
...,...,...,...,...
50,12700,780000.0000,3000.0000,0.3846
6,41860,1297500.0000,4500.0000,0.3468
17,46520,1145000.0000,3700.0000,0.3231
42,17980,433950.0000,1272.0000,0.2931


In [659]:
### Merge now with the main dataset
census_bls_permits_housecanary = census_bls_permits.merge(hc_price_and_rent, how='left', on='msa_code')
census_bls_permits_housecanary

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio
0,"Salisbury, MD",41540,0.0550,0.0153,128487.0000,0.0053,0.0034,0.1162,248500.0000,0.0720,1021.0000,0.2391,0.4109,54646.0000,"Salisbury, MD",2,41540,180800.0000,0.0290,47292.9600,0.2144,3.8000,2023.0000,4177.0000,90.0000,110.0000,607.0000,4894.0000,85.3494,1.8390,2.2477,12.4029,0.0896,415000.0000,2450.0000,0.5904
1,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,202800.0000,0.1118,44395.5200,0.3483,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,367750.0000,2251.0000,0.6121
2,"The Villages, FL",45540,0.2281,0.0466,131832.0000,0.1291,0.0353,0.1640,NaN,NaN,NaN,NaN,NaN,76923.0000,"Wildwood-The Villages, FL",2,45540,41200.0000,0.2997,41499.1200,0.0161,5.2000,2023.0000,3339.0000,0.0000,0.0000,0.0000,3339.0000,100.0000,0.0000,0.0000,0.0000,0.0434,335495.0000,2275.0000,0.6781
3,"Sherman, TX",43300,0.0954,0.0208,137008.0000,0.0861,0.0209,0.1161,281700.0000,0.7272,1112.0000,0.5129,0.3947,58923.0000,"Sherman-Denison, TX",2,43300,54300.0000,0.1082,62283.5200,0.4483,4.2000,2023.0000,1162.0000,26.0000,26.0000,1259.0000,2447.0000,47.4867,1.0625,1.0625,51.4508,0.0415,NaN,NaN,NaN
4,"Austin, TX",12420,0.2500,0.0501,2296377.0000,0.1478,0.0278,0.0567,487200.0000,0.6000,1596.0000,0.3842,0.3276,960087.0000,"Austin-Round Rock-San Marcos, TX",5,12420,1350200.0000,0.2046,63908.5200,0.1591,3.7000,2023.0000,16532.0000,244.0000,488.0000,21753.0000,38773.0000,42.6379,0.6293,1.2586,56.1035,0.0404,418000.0000,2295.0000,0.5490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,"Wooster, OH",49300,0.0197,0.0055,116680.0000,0.0066,-0.0015,0.0524,NaN,NaN,NaN,NaN,NaN,46649.0000,"Wooster, OH",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
924,"Worthington, MN",49380,-0.0288,0.0109,22194.0000,0.0156,-0.0013,0.0873,NaN,NaN,NaN,NaN,NaN,8424.0000,"Worthington, MN",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,"Yankton, SD",49460,0.0545,0.0137,23311.0000,0.0287,0.0042,0.0592,NaN,NaN,NaN,NaN,NaN,10405.0000,"Yankton, SD",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926,"Zanesville, OH",49780,0.0073,0.0185,86393.0000,0.0054,0.0005,0.0978,NaN,NaN,NaN,NaN,NaN,38348.0000,"Zanesville, OH",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now merge HPI and RPI forecasts

In [660]:
hpi = pd.read_csv("../downloaded_datasets/transferred_from_house_canary_repo/msa_outputs/msa_hpi_forecast.csv",
                  dtype={'MSA_CODE':str})
hpi['MONTH'] = pd.to_datetime(hpi['MONTH'])

# Get the current and 1-year HPI to determine growth
min_year = hpi['MONTH'].min()
one_year_from_now = min_year + pd.DateOffset(years=1)
hpi = hpi[(hpi['MONTH']==min_year) | (hpi['MONTH']==one_year_from_now)]

hpi.sort_values(["MSA_NAME","MONTH"], ascending=[True, True], inplace=True)

# Create 1-year forecast
hpi['hpi_value_pct_change'] = hpi['HPI_VALUE'].pct_change()
hpi['hpi_real_pct_change'] = hpi['HPI_REAL'].pct_change()
hpi['hpi_trend_pct_change'] = hpi['HPI_TREND'].pct_change()
hpi = hpi[hpi['MONTH']==one_year_from_now]

# Keep key columns
hpi = hpi[['MSA_CODE','hpi_value_pct_change']]

# Rename
hpi.rename(columns={'MSA_CODE':'msa_code','hpi_value_pct_change':'1-Year Price Forecast'}, 
           inplace=True)

hpi



,msa_code,1-Year Price Forecast
8580,10180,0.0382
13728,10420,0.0709
2709,10500,0.0325
9948,10540,0.0303
5820,10580,0.0634
...,...,...
11928,49420,0.0420
2673,49620,0.0500
2781,49660,0.0614
13152,49700,0.0220


In [661]:
rpi = pd.read_csv("../downloaded_datasets/transferred_from_house_canary_repo/msa_outputs/msa_rpi_forecast.csv",
                  dtype={'MSA_CODE':str})
rpi['MONTH'] = pd.to_datetime(rpi['MONTH'])

# Get the current and 1-year RPI to determine growth
min_year = rpi['MONTH'].min()
one_year_from_now = rpi['MONTH'].max()
rpi = rpi[(rpi['MONTH']==min_year) | (rpi['MONTH']==one_year_from_now)]

rpi.sort_values(["MSA_NAME","MONTH"], ascending=[True, True], inplace=True)

# Create 1-year forecast
rpi['rpi_value_pct_change'] = rpi['RPI_VALUE'].pct_change()
rpi = rpi[rpi['MONTH']==one_year_from_now]

# Keep key columns
rpi = rpi[['MSA_CODE','rpi_value_pct_change']]
rpi.rename(columns={'MSA_CODE':'msa_code','rpi_value_pct_change':'1-Year Rent Forecast'}, 
           inplace=True)

rpi



,msa_code,1-Year Rent Forecast
836,10180,0.0205
1972,10420,0.0249
1160,10580,0.0544
1403,10740,0.0245
535,10900,0.0022
...,...,...
1052,49020,0.0373
2135,49180,0.0175
1333,49340,0.0342
1843,49620,0.0421


In [662]:
### Now join the two
hpi_rpi = hpi.merge(rpi, how='left', on='msa_code')
hpi_rpi

,msa_code,1-Year Price Forecast,1-Year Rent Forecast
0,10180,0.0382,0.0205
1,10420,0.0709,0.0249
2,10500,0.0325,NaN
3,10540,0.0303,NaN
4,10580,0.0634,0.0544
...,...,...,...
379,49420,0.0420,NaN
380,49620,0.0500,0.0421
381,49660,0.0614,0.0505
382,49700,0.0220,NaN


In [663]:
### Mow merge with the main dataset
census_bls_permits_housecanary_forecasts = census_bls_permits_housecanary.merge(hpi_rpi,
    how='left', on='msa_code')
census_bls_permits_housecanary_forecasts

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast
0,"Salisbury, MD",41540,0.0550,0.0153,128487.0000,0.0053,0.0034,0.1162,248500.0000,0.0720,1021.0000,0.2391,0.4109,54646.0000,"Salisbury, MD",2,41540,180800.0000,0.0290,47292.9600,0.2144,3.8000,2023.0000,4177.0000,90.0000,110.0000,607.0000,4894.0000,85.3494,1.8390,2.2477,12.4029,0.0896,415000.0000,2450.0000,0.5904,0.0477,0.0493
1,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,202800.0000,0.1118,44395.5200,0.3483,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,367750.0000,2251.0000,0.6121,0.0218,-0.0039
2,"The Villages, FL",45540,0.2281,0.0466,131832.0000,0.1291,0.0353,0.1640,NaN,NaN,NaN,NaN,NaN,76923.0000,"Wildwood-The Villages, FL",2,45540,41200.0000,0.2997,41499.1200,0.0161,5.2000,2023.0000,3339.0000,0.0000,0.0000,0.0000,3339.0000,100.0000,0.0000,0.0000,0.0000,0.0434,335495.0000,2275.0000,0.6781,0.0147,0.0443
3,"Sherman, TX",43300,0.0954,0.0208,137008.0000,0.0861,0.0209,0.1161,281700.0000,0.7272,1112.0000,0.5129,0.3947,58923.0000,"Sherman-Denison, TX",2,43300,54300.0000,0.1082,62283.5200,0.4483,4.2000,2023.0000,1162.0000,26.0000,26.0000,1259.0000,2447.0000,47.4867,1.0625,1.0625,51.4508,0.0415,NaN,NaN,NaN,0.0284,-0.0033
4,"Austin, TX",12420,0.2500,0.0501,2296377.0000,0.1478,0.0278,0.0567,487200.0000,0.6000,1596.0000,0.3842,0.3276,960087.0000,"Austin-Round Rock-San Marcos, TX",5,12420,1350200.0000,0.2046,63908.5200,0.1591,3.7000,2023.0000,16532.0000,244.0000,488.0000,21753.0000,38773.0000,42.6379,0.6293,1.2586,56.1035,0.0404,418000.0000,2295.0000,0.5490,0.0289,-0.0369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,"Wooster, OH",49300,0.0197,0.0055,116680.0000,0.0066,-0.0015,0.0524,NaN,NaN,NaN,NaN,NaN,46649.0000,"Wooster, OH",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
924,"Worthington, MN",49380,-0.0288,0.0109,22194.0000,0.0156,-0.0013,0.0873,NaN,NaN,NaN,NaN,NaN,8424.0000,"Worthington, MN",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,"Yankton, SD",49460,0.0545,0.0137,23311.0000,0.0287,0.0042,0.0592,NaN,NaN,NaN,NaN,NaN,10405.0000,"Yankton, SD",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926,"Zanesville, OH",49780,0.0073,0.0185,86393.0000,0.0054,0.0005,0.0978,NaN,NaN,NaN,NaN,NaN,38348.0000,"Zanesville, OH",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now merge Taxes and Insurance

In [664]:
### Read in insurance
insurance = pd.read_csv("../datasets_manual_download/Insurance_and_PropTax/avg_insurance_2024.csv")
insurance.head(5)

,insurance,state
0,2064,AK
1,3798,AL
2,4675,AR
3,3063,AZ
4,1772,CA


In [665]:
### Read in proptaxes
msa_taxes = pd.read_csv(
    "../datasets_manual_download/Insurance_and_PropTax/proptaxes_by_msa.csv",
    dtype={'msa_code':str})
msa_taxes = clean_BLS_msa_names(msa_taxes)

# Merge with insurance
msa_taxes_insurance = msa_taxes.merge(insurance, how='left', on='state')

# Only keep relevant columns
msa_taxes_insurance = msa_taxes_insurance[['msa_code',
                                           'state',
                                           'metro_or_micro_area',
                                           'median_prop_taxes_by_msa','insurance']]

msa_taxes_insurance.head(5)

,msa_code,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance
0,10100,SD,Micropolitan Statistical Area,2277.0000,3970.0000
1,10140,WA,Micropolitan Statistical Area,1977.0000,2017.0000
2,10180,TX,Metropolitan Statistical Area,2256.0000,4643.0000
3,10220,OK,Micropolitan Statistical Area,866.0000,7012.0000
4,10300,MI,Micropolitan Statistical Area,2188.0000,2956.0000


In [666]:
### Now merge with main dataset
biggerscore_v1 = census_bls_permits_housecanary_forecasts.merge(
    msa_taxes_insurance, how='left', on='msa_code')

# FOR TESTING PURPOSES, CREATE A COLUMN THAT RELATES 1-YEAR HOUSEHOLD GROWTH 
# TO PERMITS AS PERCENTAGE OF TOTAL UNITS
biggerscore_v1['1-Year_HH_Growth_Minus_Percent_New_Supply'] = biggerscore_v1['1-Year Household Growth'] - biggerscore_v1['Permits_as_Percent_of_Total_Units']

biggerscore_v1

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply
0,"Salisbury, MD",41540,0.0550,0.0153,128487.0000,0.0053,0.0034,0.1162,248500.0000,0.0720,1021.0000,0.2391,0.4109,54646.0000,"Salisbury, MD",2,41540,180800.0000,0.0290,47292.9600,0.2144,3.8000,2023.0000,4177.0000,90.0000,110.0000,607.0000,4894.0000,85.3494,1.8390,2.2477,12.4029,0.0896,415000.0000,2450.0000,0.5904,0.0477,0.0493,MD,Metropolitan Statistical Area,1932.0000,2131.0000,-0.0742
1,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,202800.0000,0.1118,44395.5200,0.3483,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,367750.0000,2251.0000,0.6121,0.0218,-0.0039,SC,Metropolitan Statistical Area,725.0000,3219.0000,-0.0060
2,"The Villages, FL",45540,0.2281,0.0466,131832.0000,0.1291,0.0353,0.1640,NaN,NaN,NaN,NaN,NaN,76923.0000,"Wildwood-The Villages, FL",2,45540,41200.0000,0.2997,41499.1200,0.0161,5.2000,2023.0000,3339.0000,0.0000,0.0000,0.0000,3339.0000,100.0000,0.0000,0.0000,0.0000,0.0434,335495.0000,2275.0000,0.6781,0.0147,0.0443,FL,Metropolitan Statistical Area,2645.0000,4984.0000,0.0032
3,"Sherman, TX",43300,0.0954,0.0208,137008.0000,0.0861,0.0209,0.1161,281700.0000,0.7272,1112.0000,0.5129,0.3947,58923.0000,"Sherman-Denison, TX",2,43300,54300.0000,0.1082,62283.5200,0.4483,4.2000,2023.0000,1162.0000,26.0000,26.0000,1259.0000,2447.0000,47.4867,1.0625,1.0625,51.4508,0.0415,NaN,NaN,NaN,0.0284,-0.0033,TX,Metropolitan Statistical Area,2352.0000,4643.0000,-0.0207
4,"Austin, TX",12420,0.2500,0.0501,2296377.0000,0.1478,0.0278,0.0567,487200.0000,0.6000,1596.0000,0.3842,0.3276,960087.0000,"Austin-Round Rock-San Marcos, TX",5,12420,1350200.0000,0.2046,63908.5200,0.1591,3.7000,2023.0000,16532.0000,244.0000,488.0000,21753.0000,38773.0000,42.6379,0.6293,1.2586,56.1035,0.0404,418000.0000,2295.0000,0.5490,0.0289,-0.0369,TX,Metropolitan Statistical Area,6517.0000,4643.0000,0.0097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,"Wooster, OH",49300,0.0197,0.0055,116680.0000,0.0066,-0.0015,0.0524,NaN,NaN,NaN,NaN,NaN,46649.0000,"Wooster, OH",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OH,Micropolitan Statistical Area,2229.0000,2613.0000,NaN
924,"Worthington, MN",49380,-0.0288,0.0109,22194.0000,0.0156,-0.0013,0.0873,NaN,NaN,NaN,NaN,NaN,8424.0000,"Worthington, MN",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MN,Micropolitan Statistical Area,1418.0000,2999.0000,NaN
925,"Yankton, SD",49460,0.0545,0.0137,23311.0000,0.0287,0.0042,0.0592,NaN,NaN,NaN,NaN,NaN,10405.0000,"Yankton, SD",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SD,Micropolitan Statistical Area,1971.0000,3970.0000,NaN
926,"Zanesville, OH",49780,0.0073,0.0185,86393.0000,0.0054,0.0005,0.0978,NaN,NaN,NaN,NaN,NaN,38348.0000,"Zanesville, OH",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### EDA on V1 Dataset, Look for missing data

In [667]:
usa_msa = biggerscore_v1[
    (biggerscore_v1['metro_or_micro_area'].str.contains("Metro"))
    & (biggerscore_v1['state']!="PR")
    ].copy()

usa_msa.sort_values("5-Year Population Growth", ascending=False).head(20)
usa_msa[usa_msa['Population_Size_Category']>=4].sort_values("5-Year Population Growth", ascending=False).head(20)

usa_msa[usa_msa['Population_Size_Category']>=3].sort_values("5-Year Income Growth", ascending=False).head(20)

usa_msa[usa_msa['Population_Size_Category']>=3].sort_values(
    "Permits_as_Percent_of_Total_Units", ascending=False).head(5)


# usa_msa.sort_values("Vacancy_Rate", ascending=True).head(20)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply
1,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,202800.0000,0.1118,44395.5200,0.3483,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,367750.0000,2251.0000,0.6121,0.0218,-0.0039,SC,Metropolitan Statistical Area,725.0000,3219.0000,-0.0060
4,"Austin, TX",12420,0.2500,0.0501,2296377.0000,0.1478,0.0278,0.0567,487200.0000,0.6000,1596.0000,0.3842,0.3276,960087.0000,"Austin-Round Rock-San Marcos, TX",5,12420,1350200.0000,0.2046,63908.5200,0.1591,3.7000,2023.0000,16532.0000,244.0000,488.0000,21753.0000,38773.0000,42.6379,0.6293,1.2586,56.1035,0.0404,418000.0000,2295.0000,0.5490,0.0289,-0.0369,TX,Metropolitan Statistical Area,6517.0000,4643.0000,0.0097
6,"Lakeland, FL",29460,0.1657,0.0433,736229.0000,0.1287,0.0324,0.1746,296300.0000,0.8369,1192.0000,0.5165,0.4023,320023.0000,"Lakeland-Winter Haven, FL",4,29460,278200.0000,0.1773,53181.4400,0.2650,4.6000,2023.0000,9209.0000,124.0000,135.0000,3169.0000,12513.0000,73.5955,0.9910,1.0789,25.3257,0.0391,315770.0000,2195.0000,0.6951,0.0145,0.0008,FL,Metropolitan Statistical Area,1398.0000,4984.0000,0.0042
9,"Raleigh, NC",39580,0.1372,0.0241,1420825.0000,0.1153,0.0209,0.0781,439500.0000,0.6424,1451.0000,0.4990,0.3301,581802.0000,"Raleigh-Cary, NC",5,39580,751300.0000,0.1619,60454.6800,0.2049,3.8000,2023.0000,12147.0000,82.0000,85.0000,8387.0000,20619.0000,58.9117,0.3977,0.4122,40.6761,0.0354,460525.0000,2195.0000,0.4766,0.0236,-0.0325,NC,Metropolitan Statistical Area,2668.0000,3398.0000,-0.0113
10,"Crestview, FL",18880,0.1255,0.0255,288639.0000,0.1057,0.0211,0.2740,374300.0000,0.5687,1458.0000,0.5952,0.3895,158152.0000,"Crestview-Fort Walton Beach-Destin, FL",3,18880,134600.0000,0.1226,55722.1600,0.3196,3.3000,2023.0000,2814.0000,52.0000,72.0000,2710.0000,5596.0000,50.2859,0.9292,1.2866,48.4274,0.0354,375470.0000,2495.0000,0.6645,0.0081,-0.0337,FL,Metropolitan Statistical Area,1623.0000,4984.0000,-0.0099


### To begin the weighting, start by creating an algorithm that works for me (Austin)

In [668]:
# I'll start by only keeping metropolitan areas within the USA (no puerto rico, no micro areas)
usa_msa = biggerscore_v1[
    (biggerscore_v1['metro_or_micro_area'].str.contains("Metro"))
    & (biggerscore_v1['state']!="PR")
    ].copy()

usa_msa[
    (usa_msa['5-Year Household Growth']>=usa_msa['5-Year Household Growth'].quantile(0.25))
    # & (usa_msa['1-Year Household Growth']>=usa_msa['1-Year Household Growth'].quantile(0.25))
    & (usa_msa['5-Year Population Growth']>=usa_msa['5-Year Population Growth'].quantile(0.25))
    # & (usa_msa['1-Year Population Growth']>=usa_msa['1-Year Population Growth'].quantile(0.25))
    & (usa_msa['5-Year Job Growth']>=usa_msa['5-Year Job Growth'].quantile(0.25))
    & (usa_msa['Income']>=usa_msa['Income'].quantile(0.25))
    & (usa_msa['5-Year Income Growth']>=0)
    & (usa_msa['Unemployment_Rate']<=usa_msa['Unemployment_Rate'].quantile(0.75))
    & (usa_msa['median_prop_taxes_by_msa']<=usa_msa['median_prop_taxes_by_msa'].quantile(0.75))
    # & (usa_msa['insurance']<=usa_msa['insurance'].quantile(0.9))
    & (usa_msa['Vacancy_Rate']<=usa_msa['Vacancy_Rate'].quantile(0.5))
    & (usa_msa['Population_Size_Category']>=2)
    # & (usa_msa['Permits_as_Percent_of_Total_Units']<=usa_msa['Permits_as_Percent_of_Total_Units'].quantile(0.9))
    & (usa_msa['5-Year Rent Growth']>=usa_msa['5-Year Rent Growth'].quantile(0.25))
    # & (usa_msa['5-Year Price Growth']>=usa_msa['5-Year Price Growth'].quantile(0.25))

    # ].sort_values("1-Year Price Forecast", ascending=False).head(50)
    ].sort_values("1-Year_HH_Growth_Minus_Percent_New_Supply", ascending=False).head(50)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply
45,"Logan, UT",30860,0.1283,0.0333,148804.0000,0.1154,0.0210,0.0588,466900.0000,0.8245,1054.0000,0.6067,0.2257,49287.0000,"Logan, UT-ID",2,30860,70200.0000,0.1359,47922.1600,0.0465,3.5000,2023.0000,693.0000,4.0000,8.0000,339.0000,1040.0000,66.6346,0.3846,0.7692,32.5962,0.0211,NaN,NaN,NaN,0.0668,NaN,UT,Metropolitan Statistical Area,1596.0000,2161.0000,0.0122
17,"Provo, UT",39340,0.2001,0.0431,677964.0000,0.1546,0.0274,0.0423,566900.0000,0.7174,1415.0000,0.5150,0.2496,199201.0000,"Provo-Orem-Lehi, UT",4,39340,312400.0000,0.1644,55432.0000,0.0785,4.0000,2023.0000,4663.0000,38.0000,172.0000,1346.0000,6181.0000,75.4409,0.6148,2.7827,21.7764,0.0310,NaN,NaN,NaN,0.0287,NaN,UT,Metropolitan Statistical Area,1817.0000,2161.0000,0.0120
204,"Albany, OR",10540,0.0795,0.0169,128598.0000,0.0621,0.0110,0.0422,398300.0000,0.5663,1181.0000,0.3927,0.2965,52145.0000,"Albany, OR",2,10540,48200.0000,0.0478,58515.6000,0.3933,4.5000,2023.0000,239.0000,52.0000,59.0000,79.0000,377.0000,63.3952,13.7931,15.6499,20.9549,0.0072,NaN,NaN,NaN,0.0303,NaN,OR,Metropolitan Statistical Area,2793.0000,2185.0000,0.0097
203,"Harrisburg, PA",25420,0.0558,0.0168,593318.0000,0.0501,0.0101,0.0612,259700.0000,0.3770,969.0000,0.2344,0.3731,255664.0000,"Harrisburg-Carlisle, PA",4,25420,368700.0000,0.0392,58116.7600,0.1988,3.8000,2023.0000,1213.0000,20.0000,70.0000,581.0000,1864.0000,65.0751,1.0730,3.7554,31.1695,0.0073,247500.0000,1995.0000,0.8061,0.0567,0.0370,PA,Metropolitan Statistical Area,2958.0000,2130.0000,0.0095
200,"Columbia, MO",17860,0.0598,0.0164,211078.0000,0.0431,0.0065,0.0885,259100.0000,0.3481,904.0000,0.2970,0.3489,91879.0000,"Columbia, MO",2,17860,106800.0000,0.0876,47276.8400,0.2120,3.2000,2023.0000,609.0000,4.0000,4.0000,68.0000,681.0000,89.4273,0.5874,0.5874,9.9853,0.0074,NaN,NaN,NaN,0.0596,NaN,MO,Metropolitan Statistical Area,1943.0000,4114.0000,0.0090
206,"Albuquerque, NM",10740,0.0651,0.0157,915968.0000,0.0121,0.0028,0.0709,297100.0000,0.4752,1046.0000,0.3497,0.3521,395967.0000,"Albuquerque, NM",4,10740,423900.0000,0.0627,47620.5600,0.1029,4.3000,2023.0000,2057.0000,0.0000,0.0000,777.0000,2834.0000,72.5829,0.0000,0.0000,27.4171,0.0072,342495.0000,2345.0000,0.6847,0.0378,0.0245,NM,Metropolitan Statistical Area,2273.0000,3489.0000,0.0086
71,"Fort Collins, CO",22660,0.1406,0.0253,359363.0000,0.0858,0.0132,0.0643,574900.0000,0.4434,1575.0000,0.2560,0.2740,159083.0000,"Fort Collins-Loveland, CO",3,22660,188700.0000,0.0789,58520.8000,0.2373,3.9000,2023.0000,1289.0000,12.0000,39.0000,1358.0000,2686.0000,47.9896,0.4468,1.4520,50.5585,0.0169,NaN,NaN,NaN,0.0454,NaN,CO,Metropolitan Statistical Area,2281.0000,4662.0000,0.0084
219,"Cheyenne, WY",16940,0.0770,0.0138,100316.0000,0.0339,0.0041,0.0672,357300.0000,0.4495,991.0000,0.2837,0.2774,44389.0000,"Cheyenne, WY",2,16940,49100.0000,0.0294,51745.2000,0.3020,3.4000,2023.0000,165.0000,14.0000,114.0000,8.0000,287.0000,57.4913,4.8780,39.7213,2.7875,0.0065,NaN,NaN,NaN,0.0173,NaN,WY,Metropolitan Statistical Area,1549.0000,2490.0000,0.0074
125,"Rochester, MN",40340,0.0766,0.0188,225911.0000,0.0533,0.0070,0.0602,308900.0000,0.4170,1132.0000,0.4203,0.3665,97342.0000,"Rochester, MN",2,4

### Now begin creating the first version of weights

In [703]:
usa_msa_weighted = usa_msa.copy()
usa_msa_weighted = usa_msa_weighted.dropna()
usa_msa_weighted

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply
0,"Salisbury, MD",41540,0.0550,0.0153,128487.0000,0.0053,0.0034,0.1162,248500.0000,0.0720,1021.0000,0.2391,0.4109,54646.0000,"Salisbury, MD",2,41540,180800.0000,0.0290,47292.9600,0.2144,3.8000,2023.0000,4177.0000,90.0000,110.0000,607.0000,4894.0000,85.3494,1.8390,2.2477,12.4029,0.0896,415000.0000,2450.0000,0.5904,0.0477,0.0493,MD,Metropolitan Statistical Area,1932.0000,2131.0000,-0.0742
1,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,202800.0000,0.1118,44395.5200,0.3483,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,367750.0000,2251.0000,0.6121,0.0218,-0.0039,SC,Metropolitan Statistical Area,725.0000,3219.0000,-0.0060
4,"Austin, TX",12420,0.2500,0.0501,2296377.0000,0.1478,0.0278,0.0567,487200.0000,0.6000,1596.0000,0.3842,0.3276,960087.0000,"Austin-Round Rock-San Marcos, TX",5,12420,1350200.0000,0.2046,63908.5200,0.1591,3.7000,2023.0000,16532.0000,244.0000,488.0000,21753.0000,38773.0000,42.6379,0.6293,1.2586,56.1035,0.0404,418000.0000,2295.0000,0.5490,0.0289,-0.0369,TX,Metropolitan Statistical Area,6517.0000,4643.0000,0.0097
5,"Punta Gorda, FL",39460,0.1307,0.0232,189900.0000,0.0962,0.0274,0.2395,365900.0000,0.7711,1331.0000,0.5074,0.3638,111330.0000,"Punta Gorda, FL",2,39460,56400.0000,0.1394,41264.6000,0.1781,4.2000,2023.0000,3804.0000,284.0000,395.0000,230.0000,4429.0000,85.8885,6.4123,8.9185,5.1930,0.0398,332000.0000,2570.0000,0.7741,0.0126,-0.2086,FL,Metropolitan Statistical Area,2064.0000,4984.0000,-0.0166
6,"Lakeland, FL",29460,0.1657,0.0433,736229.0000,0.1287,0.0324,0.1746,296300.0000,0.8369,1192.0000,0.5165,0.4023,320023.0000,"Lakeland-Winter Haven, FL",4,29460,278200.0000,0.1773,53181.4400,0.2650,4.6000,2023.0000,9209.0000,124.0000,135.0000,3169.0000,12513.0000,73.5955,0.9910,1.0789,25.3257,0.0391,315770.0000,2195.0000,0.6951,0.0145,0.0008,FL,Metropolitan Statistical Area,1398.0000,4984.0000,0.0042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,"Reading, PA",39740,0.0474,0.0069,428483.0000,0.0312,0.0036,0.0557,261200.0000,0.4043,996.0000,0.3533,0.3813,170678.0000,"Reading, PA",3,39740,180800.0000,0.0011,63993.8000,0.3461,4.3000,2023.0000,231.0000,12.0000,16.0000,105.0000,352.0000,65.6250,3.4091,4.5455,29.8295,0.0021,291000.0000,2050.0000,0.7045,0.0631,0.0401,PA,Metropolitan Statistical Area,3893.0000,2130.0000,0.0049
357,"Atlantic City, NJ",12100,0.0669,0.0181,369795.0000,0.0063,0.0012,0.3519,350000.0000,0.5960,1132.0000,0.2358,0.3234,231311.0000,"Atlantic City-Hammonton, NJ",3,12100,133300.0000,-0.0191,49566.9200,0.1094,6.8000,2023.0000,299.0000,44.0000,44.0000,40.0000,383.0000,78.0679,11.4883,11.4883,10.4439,0.0017,370000.0000,2350.0000,0.6351,0.0806,0.0107,NJ,Metropolitan Statistical Area,6373.0000,1894.0000,0.0165
361,"Youngstown, OH",49660,-0.0111,0.0023,429728.0000,-0.0126,-0.0035,0.0990,149700.0000,0.3051,626.0000,0.1636,0.4182,202778.0000,"Youngstown-Warre

In [707]:
usa_msa_weighted = usa_msa.copy()

# Drop rows where key rows are is empty
usa_msa_weighted = usa_msa_weighted[usa_msa_weighted['Jobs'].notna()]
usa_msa_weighted = usa_msa_weighted[usa_msa_weighted['1-Year_HH_Growth_Minus_Percent_New_Supply'].notna()]


# Make a function for this where the input is the column name
def basic_column_ranking(
        df, 
        col_name, 
        ascending=True):
    
    df = df.sort_values(col_name, ascending=ascending).reset_index(drop=True)
    df[f'RANK_{col_name}'] = df.index + 1

    return df

# Create the base rankings
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "5-Year Household Growth")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "5-Year Population Growth")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "5-Year Job Growth")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "Income")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "5-Year Income Growth")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "Unemployment_Rate", ascending=False)
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "1-Year_HH_Growth_Minus_Percent_New_Supply")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "ACS_1_Year_Rent-Price_Ratio")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "1-Year Price Forecast")
# usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "1-Year Rent Forecast")
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "insurance", ascending=False)
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "median_prop_taxes_by_msa", ascending=False)
usa_msa_weighted = basic_column_ranking(usa_msa_weighted, "Vacancy_Rate", ascending=False)

usa_msa_weighted.head(5)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_5-Year Household Growth,RANK_5-Year Population Growth,RANK_5-Year Job Growth,RANK_Income,RANK_5-Year Income Growth,RANK_Unemployment_Rate,RANK_1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_ACS_1_Year_Rent-Price_Ratio,RANK_1-Year Price Forecast,RANK_insurance,RANK_median_prop_taxes_by_msa,RANK_Vacancy_Rate
0,"Barnstable Town, MA",12700,0.0522,0.0184,229436.0000,0.0726,0.0066,0.3944,634700.0000,0.5556,1398.0000,0.2572,0.2203,165068.0000,"Barnstable Town, MA",2,70900,115900.0000,-0.0508,54119.5200,0.2276,4.1000,2023.0000,418.0000,12.0000,25.0000,104.0000,547.0000,76.4168,2.1938,4.5704,19.0128,0.0033,780000.0000,3000.0000,0.3846,0.0487,0.0041,MA,Metropolitan Statistical Area,3658.0000,1998.0000,0.0151,198,306,10,218,218,184,350,9,248,324,66,1
1,"Atlantic City, NJ",12100,0.0669,0.0181,369795.0000,0.0063,0.0012,0.3519,350000.0000,0.5960,1132.0000,0.2358,0.3234,231311.0000,"Atlantic City-Hammonton, NJ",3,12100,133300.0000,-0.0191,49566.9200,0.1094,6.8000,2023.0000,299.0000,44.0000,44.0000,40.0000,383.0000,78.0679,11.4883,11.4883,10.4439,0.0017,370000.0000,2350.0000,0.6351,0.0806,0.0107,NJ,Metropolitan Statistical Area,6373.0000,1894.0000,0.0165,238,111,42,138,88,12,357,120,357,330,13,2
2,"Naples, FL",34940,0.1349,0.0199,380221.0000,0.0657,0.0199,0.3178,595500.0000,0.6108,1867.0000,0.6420,0.3135,229814.0000,"Naples-Marco Island, FL",3,34940,173600.0000,0.1581,50788.4000,0.1160,3.7000,2023.0000,2923.0000,30.0000,106.0000,589.0000,3618.0000,80.7905,0.8292,2.9298,16.2797,0.0157,710000.0000,4200.0000,0.5915,0.0259,-0.1459,FL,Metropolitan Statistical Area,2807.0000,4984.0000,0.0041,346,294,357,155,96,258,217,100,79,16,129,3
3,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,202800.0000,0.1118,44395.5200,0.3483,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,367750.0000,2251.0000,0.6121,0.0218,-0.0039,SC,Metropolitan Statistical Area,725.0000,3219.0000,-0.0060,349,364,330,72,327,42,29,177,52,172,358,4
4,"Daphne, AL",19300,0.1927,0.0414,233420.0000,0.1478,0.0277,0.2742,307000.0000,0.5841,1089.0000,0.3105,0.3547,125113.0000,"Daphne-Fairhope-Foley, AL",2,19300,89800.0000,0.1018,43076.2800,0.2457,3.1000,2023.0000,3316.0000,16.0000,32.0000,1015.0000,4363.0000,76.0028,0.3667,0.7334,23.2638,0.0349,360432.0000,2000.0000,0.5549,0.0321,-0.0105,AL,Metropolitan Statistical Area,718.0000,3798.0000,0.0066,364,362,316,51,245,329,268,194,126,108,359,5


In [723]:
usa_msa_weighted.to_csv("outputs/ranked_msas.csv", index=False)

In [724]:
### Create a dictionary to hold the future weights of the RANK columns
rank_weights = {
    'RANK_5-Year Household Growth': 1,
    'RANK_5-Year Population Growth': 1,
    'RANK_5-Year Job Growth': 3,
    'RANK_Income': 2,
    'RANK_5-Year Income Growth': 1,
    'RANK_Unemployment_Rate': 1,
    'RANK_1-Year_HH_Growth_Minus_Percent_New_Supply': 2,
    'RANK_ACS_1_Year_Rent-Price_Ratio': 3,
    'RANK_1-Year Price Forecast': 2,
    # 'RANK_1-Year Rent Forecast': 1,
    'RANK_insurance': 3,
    'RANK_median_prop_taxes_by_msa': 3,
    'RANK_Vacancy_Rate': 2,
    'Population_Size_Category': 250
}

### Create the basic sums of the ranks
usa_msa_weighted['Total_Rank_Sum'] = 0
for col in rank_weights:
    usa_msa_weighted['Total_Rank_Sum'] += usa_msa_weighted[col] * rank_weights[col]

usa_msa_weighted.sort_values("Total_Rank_Sum", ascending=False).head(10)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_5-Year Household Growth,RANK_5-Year Population Growth,RANK_5-Year Job Growth,RANK_Income,RANK_5-Year Income Growth,RANK_Unemployment_Rate,RANK_1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_ACS_1_Year_Rent-Price_Ratio,RANK_1-Year Price Forecast,RANK_insurance,RANK_median_prop_taxes_by_msa,RANK_Vacancy_Rate,Total_Rank_Sum
344,"Salt Lake City, UT",41620,0.1233,0.0211,1254675.0000,0.0723,0.0080,0.0535,547500.0000,0.6631,1491.0000,0.4632,0.2723,454070.0000,"Salt Lake City-Murray, UT",5,41620,844000.0000,0.1189,67193.8800,0.2873,3.8000,2023.0000,3163.0000,96.0000,390.0000,5682.0000,9235.0000,34.2501,1.0395,4.2231,61.5268,0.0203,600000.0000,2815.0000,0.4692,0.0469,NaN,UT,Metropolitan Statistical Area,2365.0000,2161.0000,0.0007,332,304,336,352,284,238,129,42,233,277,175,345,7016
360,"Boise City, ID",14260,0.1533,0.0334,771602.0000,0.1392,0.0279,0.0455,471700.0000,0.7840,1379.0000,0.6320,0.2923,298787.0000,"Boise City, ID",4,14260,409400.0000,0.1768,56797.5200,0.3753,3.6000,2023.0000,6508.0000,38.0000,268.0000,3115.0000,9891.0000,65.7972,0.3842,2.7095,31.4933,0.0331,492495.0000,2612.0000,0.5304,0.0560,-0.0039,ID,Metropolitan Statistical Area,2229.0000,2449.0000,0.0003,356,361,361,268,337,275,123,69,288,246,190,361,7007
298,"Richmond, VA",40060,0.0967,0.0202,1316145.0000,0.0552,0.0099,0.0661,357600.0000,0.4531,1303.0000,0.3996,0.3644,553455.0000,"Richmond, VA",5,40060,729000.0000,0.0611,57579.0800,0.2395,3.6000,2023.0000,4590.0000,224.0000,232.0000,5151.0000,9973.0000,46.0243,2.2461,2.3263,51.6495,0.0180,399975.0000,2395.0000,0.5988,0.0641,0.0297,VA,Metropolitan Statistical Area,2500.0000,2694.0000,0.0021,299,268,240,287,239,276,164,213,326,221,160,299,6986
114,"Charleston, SC",16700,0.1324,0.0266,803398.0000,0.0796,0.0170,0.1206,384800.0000,0.5216,1505.0000,0.5095,0.3911,362156.0000,"Charleston-North Charleston, SC",4,16700,433600.0000,0.1429,58421.4800,0.2334,4.6000,2023.0000,6184.0000,82.0000,316.0000,2073.0000,8573.0000,72.1334,0.9565,3.6860,24.1806,0.0237,423190.0000,2800.0000,0.6616,0.0537,0.0500,SC,Metropolitan Statistical Area,1573.0000,3219.0000,0.0030,341,317,353,299,226,115,188,265,277,176,269,115,6946
314,"Harrisburg, PA",25420,0.0558,0.0168,593318.0000,0.0501,0.0101,0.0612,259700.0000,0.3770,969.0000,0.2344,0.3731,255664.0000,"Harrisburg-Carlisle, PA",4,25420,368700.0000,0.0392,58116.7600,0.1988,3.8000,2023.0000,1213.0000,20.0000,70.0000,581.0000,1864.0000,65.0751,1.0730,3.7554,31.1695,0.0073,247500.0000,1995.0000,0.8061,0.0567,0.0370,PA,Metropolitan Statistical Area,2958.0000,2130.0000,0.0095,214,258,199,296,180,243,317,231,293,288,119,315,6848
350,"Ogden, UT",36260,0.1059,0.0221,638359.0000,0.0818,0.0127,0.0501,487400.0000,0.7235,1364.0000,0.5660,0.2799,216503.0000,"Ogden, UT",4,36260,303700.0000,0.1256,51878.3200,0.1879,3.7000,2023.0000,1871.0000,2.0000,158.0000,1371.0000,3400.0000,55.0294,0.0588,4.6471,40.3235,0.0157,NaN,NaN,NaN,0.0544,NaN,UT,Metropolitan Statistical Area,2115.0000,2161.0000,0.0064,311,321,343,165,171,261,266,52,279,275,197,351,6787
178,"Las Vegas, NV",29820,0.1100,0.0197,2265926.0000,0.0727,0.0156,0.0985,437900.0000,0.5279,1450.0000,0.4872,0.3311,923275.0000,

In [670]:
usa_msa[usa_msa['msa_name'].str.contains("Fayetteville")]

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply
18,"Fayetteville, AR",22220,0.1247,0.0287,550596.0000,0.1204,0.0233,0.0845,342100.0000,0.9706,1001.0000,0.4918,0.2926,223045.0000,"Fayetteville-Springdale-Rogers, AR",4,22220,312800.0000,0.1620,56077.8400,0.3701,2.6000,2023.0000,5349.0000,428.0000,511.0000,1055.0000,6915.0000,77.3536,6.1894,7.3897,15.2567,0.0310,338745.0000,1995.0000,0.5889,0.0379,0.0295,AR,Metropolitan Statistical Area,1442.0000,4675.0000,-0.0023
87,"Fayetteville, NC",22180,0.0236,0.0165,387819.0000,0.0064,0.0054,0.1092,216900.0000,0.5638,1028.0000,0.4398,0.4740,163024.0000,"Fayetteville, NC",3,22180,136400.0000,0.0476,42458.5200,0.3146,5.7000,2023.0000,2542.0000,4.0000,4.0000,30.0000,2576.0000,98.6801,0.1553,0.1553,1.1646,0.0158,301750.0000,1800.0000,0.5965,0.0315,-0.0100,NC,Metropolitan Statistical Area,1665.0000,3398.0000,0.0007
